# Streaming ASR
In this tutorial, we will look at one way to use one of NeMo's pretrained Conformer-CTC models for streaming inference. We will first look at some use cases where we may need streaming inference and then we will work towards developing a method for transcribing a long audio file using streaming.

# Why Stream?
Streaming inference may be needed in one of the following scenarios:
* Real-time or close to real-time inference for live transcriptions
* Offline transcriptions of very long audio

In this tutorial, we will mainly focus on streaming for handling long form audio and close to real-time inference with CTC based models. For training ASR models we usually use short segments of audio (<20s) that may be smaller chunks of a long audio that is aligned with the transcriptions and segmented into smaller chunks (see [tools/](https://github.com/NVIDIA/NeMo/tree/main/tools) for some great tools to do this). For running inference on long audio files we are restricted by the available GPU memory that dictates the maximum length of audio that can be transcribed in one inference call. We will take a look at one of the ways to overcome this restriction using NeMo's Conformer-CTC ASR model.

# Conformer-CTC
Conformer-CTC models distributed with NeMo use a combination of self-attention and convolution modules to achieve the best of the two approaches, the self-attention layers can learn the global interaction while the convolutions efficiently capture the local correlations. Use of self-attention layers comes with a cost of increased memory usage at a quadratic rate with the sequence length. That means that transcribing long audio files with Conformer-CTC models needs streaming inference to break up the audio into smaller chunks. We will develop one method to do such inference through the course of this tutorial.

# Data
To demonstrate transcribing a long audio file we will use utterances from the dev-clean set of the [mini Librispeech corpus](https://www.openslr.org/31/).  

Let's create a long audio that is about 15 minutes long by concatenating audio from dev-clean  and also create the corresponding concatenated transcript.

In [22]:
!pip install moviepy


In [1]:
# new_duration, ref_transcript = concat_audio(manifest, 15*60)

concat_audio_path = "/external2/datasets/soccer/final.wav"

# !ffmpeg -t {new_duration} -safe 0 -f concat -i concat_file.txt -c copy -t {new_duration} {concat_audio_path} -y
# print("Finished concatenating audio file!")

# Streaming with CTC based models
Now let's try to transcribe the long audio file created above using a conformer-large model.

In [2]:
import torch
import nemo.collections.asr as nemo_asr
import contextlib
import gc

In [3]:
device = 'cpu'

We are mainly concerned about decoding on the GPU in this tutorial. CPU decoding may be able to handle longer files but would also not be as fast as GPU decoding. Let's check if we can run transcribe() on the long audio file that we created above.

In [57]:
# Clear up memory
torch.cuda.empty_cache()
gc.collect()
map_location = torch.device('cpu')

model = nemo_asr.models.ASRModel.restore_from("/external2/karan_exp/experiments/finetune-multidomain-alllang-100k-ctc/2024-07-18_00-21-12/checkpoints/finetune-multidomain-alllang-100k-ctc.nemo", map_location=map_location)
# device = 'cpu'  # You can transcribe even longer samples on the CPU, though it will take much longer !
model = model.to(device)

[NeMo I 2024-07-20 08:42:53 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1600 tokens


[NeMo W 2024-07-20 08:42:53 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/ksingla/workspace/PromptingNemo/data_v2/synthetic/processed/train.json
    sample_rate: 16000
    batch_size: 12
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2024-07-20 08:42:53 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/ksingla/workspace/PromptingNemo/data_v2/synthetic/processed/valid.json
    sample_rat

[NeMo I 2024-07-20 08:42:53 features:289] PADDING: 0
[NeMo I 2024-07-20 08:42:56 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external2/karan_exp/experiments/finetune-multidomain-alllang-100k-ctc/2024-07-18_00-21-12/checkpoints/finetune-multidomain-alllang-100k-ctc.nemo.


The call to transcribe() below should fail with a "CUDA out of memory" error when run on a GPU with 32 GB memory.

# Buffer mechanism for streaming long audio files
One way to transcribe long audio with a Conformer-CTC model would be to split the audio into consecutive smaller chunks and running inference on each chunk. Care should be taken to have enough context for audio at either edges for accurate transcription. Let's introduce some terminology here to help us navigate the rest of this tutorial. 

* Buffer size is the length of audio on which inference is run
* Chunk size is the length of new audio that is added to the buffer.

An audio buffer is made up of a chunk of audio with some padded audio from previous chunk. In order to make the best predictions with enough context for the beginning and end portions of the buffer, we only collect tokens for the middle portion of the buffer of length equal to the size of each chunk.  

Let's suppose that the maximum length of audio that can be transcribed with conformer-large model is 20s, then we can use 20s as the buffer size and use 15s (for example) as the chunk size, so one hour of audio is broken into 240 chunks of 15s each. Let's take a look at a few audio buffers that may be created for this audio.

In [58]:
# A simple iterator class to return successive chunks of samples
class AudioChunkIterator():
    def __init__(self, samples, frame_len, sample_rate):
        self._samples = samples
        self._chunk_len = chunk_len_in_secs*sample_rate
        self._start = 0
        self.output=True
   
    def __iter__(self):
        return self
    
    def __next__(self):
        if not self.output:
            raise StopIteration
        last = int(self._start + self._chunk_len)
        if last <= len(self._samples):
            chunk = self._samples[self._start: last]
            self._start = last
        else:
            chunk = np.zeros([int(self._chunk_len)], dtype='float32')
            samp_len = len(self._samples) - self._start
            chunk[0:samp_len] = self._samples[self._start:len(self._samples)]
            self.output = False
   
        return chunk

In [59]:
# a helper function for extracting samples as a numpy array from the audio file
import soundfile as sf
def get_samples(audio_file, target_sr=16000):
    with sf.SoundFile(audio_file, 'r') as f:
        sample_rate = f.samplerate
        samples = f.read()
        if sample_rate != target_sr:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
        samples = samples.transpose()
        return samples


Let's take a look at each chunk of speech that is used for decoding.

In [60]:
import matplotlib.pyplot as plt
samples = get_samples(concat_audio_path)
sample_rate  = model.preprocessor._cfg['sample_rate'] 
chunk_len_in_secs = 1            
chunk_reader = AudioChunkIterator(samples, chunk_len_in_secs, sample_rate)
count = 0
for chunk in chunk_reader:
    count +=1
    #plt.plot(chunk)
    #plt.show()
    if count >= 30:
        break


Now, let's plot the actual buffers at each stage after a new chunk is added to the buffer. Audio buffer can be thought of as a fixed size queue with each incoming chunk added at the end of the buffer and the oldest samples removed from the beginning.

In [61]:
import numpy as np
context_len_in_secs = 1

buffer_len_in_secs = chunk_len_in_secs + 2* context_len_in_secs

buffer_len = sample_rate*buffer_len_in_secs
sampbuffer = np.zeros([buffer_len], dtype=np.float32)

chunk_reader = AudioChunkIterator(samples, chunk_len_in_secs, sample_rate)
chunk_len = sample_rate*chunk_len_in_secs
count = 0
for chunk in chunk_reader:
    count +=1
    sampbuffer[:-chunk_len] = sampbuffer[chunk_len:]
    sampbuffer[-chunk_len:] = chunk
    #plt.plot(sampbuffer)
    #plt.show()
    if count >= 30:
        break

Now that we have a method to split the long audio into smaller chunks, we can now work on transcribing the individual buffers and merging the outputs to get the transcription of the whole audio.
First, we implement some helper functions to help load the buffers into the data layer.

In [62]:
from nemo.core.classes import IterableDataset

def speech_collate_fn(batch):
    """collate batch of audio sig, audio len
    Args:
        batch (FloatTensor, LongTensor):  A tuple of tuples of signal, signal lengths.
        This collate func assumes the signals are 1d torch tensors (i.e. mono audio).
    """

    _, audio_lengths = zip(*batch)

    max_audio_len = 0
    has_audio = audio_lengths[0] is not None
    if has_audio:
        max_audio_len = max(audio_lengths).item()
   
    
    audio_signal= []
    for sig, sig_len in batch:
        if has_audio:
            sig_len = sig_len.item()
            if sig_len < max_audio_len:
                pad = (0, max_audio_len - sig_len)
                sig = torch.nn.functional.pad(sig, pad)
            audio_signal.append(sig)
        
    if has_audio:
        audio_signal = torch.stack(audio_signal)
        audio_lengths = torch.stack(audio_lengths)
    else:
        audio_signal, audio_lengths = None, None

    return audio_signal, audio_lengths

# simple data layer to pass audio signal
class AudioBuffersDataLayer(IterableDataset):
    

    def __init__(self):
        super().__init__()

        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._buf_count == len(self.signal) :
            raise StopIteration
        self._buf_count +=1
        return torch.as_tensor(self.signal[self._buf_count-1], dtype=torch.float32), \
               torch.as_tensor(self.signal_shape[0], dtype=torch.int64)
        
    def set_signal(self, signals):
        self.signal = signals
        self.signal_shape = self.signal[0].shape
        self._buf_count = 0

    def __len__(self):
        return 1
    

Next we implement a class that implements transcribing audio buffers and merging the tokens corresponding to a chunk of audio within each buffer. 

For each buffer, we pick tokens corresponding to one chunk length of audio. The chunk within each buffer is chosen such that there is equal left and right context available to the audio within the chunk.

For example, if the chunk size is 1s and buffer size is 3s, we collect tokens corresponding to audio starting from 1s to 2s within each buffer. Conformer-CTC models have a model stride of 4, i.e., a token is produced for every 4 feature vectors in the time domain. MelSpectrogram features are generated once every 10 ms, so a token is produced for every 40 ms of audio.

**Note:** The inherent assumption here is that the output tokens from the model are well aligned with corresponding audio segments. This may not always be true for models trained with CTC loss, so this method of streaming inference may not always work with CTC based models. 

In [63]:
import torch
import math
import matplotlib.pyplot as plt
import cv2
import numpy as np
from torch.utils.data import DataLoader


class ChunkBufferDecoder:

    def __init__(self, asr_model, stride, chunk_len_in_secs=1, buffer_len_in_secs=3):
        self.asr_model = asr_model
        self.asr_model.eval()
        self.data_layer = AudioBuffersDataLayer()
        self.data_loader = DataLoader(self.data_layer, batch_size=1, collate_fn=speech_collate_fn)
        self.buffers = []
        self.all_preds = []
        self.chunk_len = chunk_len_in_secs
        self.buffer_len = buffer_len_in_secs
        assert(chunk_len_in_secs <= buffer_len_in_secs)
        
        feature_stride = asr_model._cfg.preprocessor['window_stride']
        self.model_stride_in_secs = feature_stride * stride
        self.n_tokens_per_chunk = math.ceil(self.chunk_len / self.model_stride_in_secs)
        self.blank_id = len(asr_model.decoder.vocabulary)
        self.plot = False
        self.frames = []

    @torch.no_grad()
    def transcribe_buffers(self, buffers, merge=True, plot=False):
        self.plot = plot
        self.buffers = buffers
        self.data_layer.set_signal(buffers[:])
        self._get_batch_preds()
        result = self.decode_final(merge)
        if plot:
            self.save_video('output_video.mp4')
        return result
    
    def _get_batch_preds(self):
        device = self.asr_model.device
        for batch in iter(self.data_loader):
            audio_signal, audio_signal_len = batch
            audio_signal, audio_signal_len = audio_signal.to(device), audio_signal_len.to(device)
            log_probs, encoded_len, predictions = self.asr_model(input_signal=audio_signal, input_signal_length=audio_signal_len)
            preds = torch.unbind(predictions)
            for pred in preds:
                self.all_preds.append(pred.cpu().numpy())
    
    def decode_final(self, merge=True, extra=0):
        self.unmerged = []
        self.toks_unmerged = []
        delay = math.ceil((self.chunk_len + (self.buffer_len - self.chunk_len) / 2) / self.model_stride_in_secs)
        decoded_frames = []
        all_toks = []
        for pred in self.all_preds:
            ids, toks = self._greedy_decoder(pred, self.asr_model.tokenizer)
            decoded_frames.append(ids)
            all_toks.append(toks)

        for decoded in decoded_frames:
            self.unmerged += decoded[len(decoded) - 1 - delay:len(decoded) - 1 - delay + self.n_tokens_per_chunk]
        if self.plot:
            for i, tok in enumerate(all_toks):
                self.plot_and_save_frame(self.buffers[i], tok[len(tok) - 1 - delay:len(tok) - 1 - delay + self.n_tokens_per_chunk])
            print("\nTokens collected from successive buffers before CTC merge")
            print(self.toks_unmerged)
        
        if not merge:
            return self.unmerged
        return self.greedy_merge(self.unmerged)
    
    def _greedy_decoder(self, preds, tokenizer):
        s = []
        ids = []
        for i in range(preds.shape[0]):
            if preds[i] == self.blank_id:
                s.append("_")
            else:
                pred = preds[i]
                s.append(tokenizer.ids_to_tokens([pred.item()])[0])
            ids.append(preds[i])
        return ids, s
    
    def greedy_merge(self, preds):
        decoded_prediction = []
        previous = self.blank_id
        for p in preds:
            if (p != previous or previous == self.blank_id) and p != self.blank_id:
                decoded_prediction.append(p.item())
            previous = p
        hypothesis = self.asr_model.tokenizer.ids_to_text(decoded_prediction)
        return hypothesis

    def plot_and_save_frame(self, buffer, tokens):
        plt.figure()
        plt.plot(buffer)
        plt.title(' '.join(tokens))
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        
        plt.savefig('frame.png')
        plt.close()
        
        frame = cv2.imread('frame.png')
        self.frames.append(frame)
        self.toks_unmerged += tokens
    
    def save_video(self, filename, fps=1):
        height, width, _ = self.frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video = cv2.VideoWriter(filename, fourcc, fps, (width, height))
        for frame in self.frames:
            video.write(frame)
        video.release()


To see how this chunk based decoder comes together, let's call the decoder with a few buffers we create from our long audio file.
Some interesting experiments to try would be to see how changing sizes of the chunk and the context affects transcription accuracy.  

In [64]:

chunk_len_in_secs = 4
context_len_in_secs = 2

buffer_len_in_secs = chunk_len_in_secs + 2* context_len_in_secs

n_buffers = 30

buffer_len = sample_rate*buffer_len_in_secs
sampbuffer = np.zeros([buffer_len], dtype=np.float32)

chunk_reader = AudioChunkIterator(samples, chunk_len_in_secs, sample_rate)
chunk_len = sample_rate*chunk_len_in_secs
count = 0
buffer_list = []
for chunk in chunk_reader:
    count +=1
    sampbuffer[:-chunk_len] = sampbuffer[chunk_len:]
    sampbuffer[-chunk_len:] = chunk
    buffer_list.append(np.array(sampbuffer))
   
    if count >= n_buffers:
        break


In [65]:
stride = 4 # 8 for Citrinet
asr_decoder = ChunkBufferDecoder(model, stride=stride, chunk_len_in_secs=chunk_len_in_secs, buffer_len_in_secs=buffer_len_in_secs )
transcription = asr_decoder.transcribe_buffers(buffer_list, plot=True)


Tokens collected from successive buffers before CTC merge
['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '▁', '▁', '▁', '▁', '▁', '_', 'T79', '_', '_', '_', '_', '▁', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'

In [66]:
# Final transcription after CTC merge
print(transcription)

T79_  వె రరి T79_T109 గ  చక్ T107 from T79_T104 బరरी స్ member  T79స T107 in T79_T69 Ffrun T107 after T79__ tul sap T79T9 icస్ T107 T79   స్పన ైకహ కనఫట్ T79__T104 సెన్స్ T107 టం  T79ాందస్థటాండి  మ సా మియన్ కలై T107 మం T79 యని రాషఫవ్ కిరి ల్ ఫoో T107 T79_ రా . T79_T104 మసి  స్టాో T107రననిing T107 ఉwవద T79_T104 ర్సలు  ని T79_T36 పీోపి యహన రా T107 . T79_T9స్ T79_T104 ెమberరి  య in   gaై  _ w  ం T79 in r ్రో ా .  కీఫలైేర మమై . T79_T110 ా్రియో చెక్్ెన్ హాల్ండా స్ర ్యల్స్ రো్రరిస్కరన ప్ేంన ీ T79__T62  inటట్ స్ పీస ోల్ రి T107క  ater T79_ వయా మై T107 T79 సరం ెం నించట్ బాె మినోలోా లస్ేఫల్ని కం చాచ మిసు ఉన్ఫూ T79_ ప . ba T79__T104 క్  టి  Dया T79_T62 ्ररीమన  రशो in T79_T104  T107 T79_T104 2 ఫల్ సes . T79_T9 సంగ్ షranਨ్ T107  సెంట్టాోయలరన  యస్టతదా లోకి చం T79_  యూోనెస్ _ స్వల్బాే T79_T90 రిస్స్మాస్ T107  ె T79_  స్ T79__   T79_ 1


Time to evaluate our streaming inference on the whole long file that we created.

In [ ]:
# WER calculation
from nemo.collections.asr.metrics.wer import word_error_rate
# Collect all buffers from the audio file
sampbuffer = np.zeros([buffer_len], dtype=np.float32)

chunk_reader = AudioChunkIterator(samples, chunk_len_in_secs, sample_rate)
buffer_list = []
for chunk in chunk_reader:
    sampbuffer[:-chunk_len] = sampbuffer[chunk_len:]
    sampbuffer[-chunk_len:] = chunk
    buffer_list.append(np.array(sampbuffer))

asr_decoder = ChunkBufferDecoder(model, stride=stride, chunk_len_in_secs=chunk_len_in_secs, buffer_len_in_secs=buffer_len_in_secs )
transcription = asr_decoder.transcribe_buffers(buffer_list, plot=False)
wer = word_error_rate(hypotheses=[transcription], references=[ref_transcript])

print(f"WER: {round(wer*100,2)}%")